In [1]:
import simpy
from random import seed, random, expovariate, normalvariate, triangular
import numpy as np
import pandas as pd
import math
early_send = 0

# Pharmacist

In [2]:
def pharmacist(env, name, TPA_premixing, TPA_mixing_duration, pharmacist_arrival_duration):
    with TPA_premixing.request() as req:
        yield req
        # pharmacist in ED or not? yes if between 8am and 6pm
        if ((env.now%1440)/60 > 8 and (env.now%1440)/60 < 18):
            print('%s TPA premixing at %d' % (name, env.now))
            yield env.timeout(TPA_mixing_duration)
            print('%s TPA premixing ended at %d' % (name, env.now))
        else:
            print('%s TPA premixing at %d' % (name, env.now))
            yield env.timeout(TPA_mixing_duration + pharmacist_arrival_duration)
            print('%s TPA premixing ended at %d' % (name, env.now))

## Walkin Patients Process

In [14]:
def walkin_patient(env, name, arrival_time, routine_care, TPA_premixing, ICU_duration, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage):
    # Simulate patient arrivals to the PSC (door in)
    yield env.timeout(arrival_time)
    print('%s walk in at %d' % (name, env.now))
    door_in = env.now
    
    # Triage
    print('%s Triage start at %d' % (name, env.now))
    yield env.timeout(triage_duration)
    print('%s Triage ended at %d' % (name, env.now))
    
    # ED Evaluation
    print('%s ED Evaluation start at %d' % (name, env.now))
    yield env.timeout(ED_eval_duration)
    print('%s ED Evaluation ended at %d' % (name, env.now))
    
    # Stroke identified? (90% yes, 10% no) if not (10%), then add extra time in routine care
    if random() > stroke_identified_percentage:
        with routine_care.request() as req:
            yield req
            print('%s Routine Care at %d' % (name, env.now))
            yield env.timeout(routine_care_duration)
            print('%s Routine Care ended at %d' % (name, env.now))
    
    # Stroke Code Activation
    
    # Pre-mix TPA
    env.process(pharmacist(env, name, TPA_premixing, TPA_mixing_duration, pharmacist_arrival_duration))
    
    # early send at node 1 (after stroke code activation)
    if early_send == 1:
        # Ambulance Contact and Arrival
        print('%s Ambulance contact at %d' % (name, env.now))
        yield env.timeout(ambulance_arrival_duration)
        print('%s Ambulance arrived at %d' % (name, env.now))
        # CSC (door out)
        print('%s Patient send to CSC at %d' % (name, env.now))
        door_out = env.now
        print('%s DIDO:' % name, door_out - door_in)
        DIDO_times.append(door_out - door_in)
        
    # no early send
    elif early_send == 0:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        # Ischemic Stroke 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # TPA (5% gets TPA)
            if random() < TPA_decision_percentage:
                if random() <  TPA_consent_percentage:
                    # TPA long consent duration (90%, 10% don't take time)
                    if random() < TPA_long_consent_percentage:
                        print('%s TPA consent at %d' % (name, env.now))
                        yield env.timeout(TPA_consent_duration)
                        print('%s TPA consent ended at %d' % (name, env.now))

                    # pharmacist in ED or not? yes if between 8am and 6pm
                    if ((env.now%1440)/60 > 8 and (env.now%1440)/60 < 18):
                        print('%s TPA mixing at %d' % (name, env.now))
                        yield env.timeout(TPA_mixing_duration)
                        print('%s TPA mixing ended and injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)
                    else:
                        print('%s TPA mixing at %d' % (name, env.now))
                        yield env.timeout(TPA_mixing_duration + pharmacist_arrival_duration)
                        print('%s TPA mixing ended and injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)
                    
                    # CTA decision (47% yes)
                    CTA = 0
                    if random() < CTA_decision_percentage:
                        CTA = 1
                        print('%s CTA Scan at %d' % (name, env.now))
                        yield env.timeout(CTA_duration)
                        print('%s CTA Scan ended at %d' % (name, env.now))

                    # transfer decision (15% yes)
                    if random() < transfer_decision_percentage:
                        if CTA:
                            # Ambulance Contact and Arrival
                            print('%s Ambulance contact at %d' % (name, env.now))
                            yield env.timeout(ambulance_arrival_duration)
                            print('%s Ambulance arrived at %d' % (name, env.now))

                        # CSC (door out)
                        print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in) 
                    else:
                        # ICU
                        door_out = env.now
                        print('%s ICU out time:' % name, door_out - door_in)
                        print('%s ICU at %d' % (name, env.now))
                        ICU_in = env.now
                        yield env.timeout(ICU_duration)
                        ICU_out = env.now
                        print('%s ICU ended at %d' % (name, env.now))  
                        ICU_times.append(ICU_out - ICU_in) 

                else: # (10% don't consent)
                    # Ambulance Contact and Arrival
                    print('%s Ambulance contact at %d' % (name, env.now))
                    yield env.timeout(ambulance_arrival_duration)
                    print('%s Ambulance arrived at %d' % (name, env.now))

                    # CSC (door out)
                    print('%s Patient send to CSC at %d' % (name, env.now))
                    door_out = env.now
                    print('%s DIDO:' % name, door_out - door_in)
                    DIDO_times.append(door_out - door_in)
                
    # early send at node 2 (after telestroke)            
    elif early_send == 2:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        # Ischemic Stroke 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # IS DIDO times
            is_DIDO_times.append(door_out - door_in)
            
    # early send at node 3 (after TPA)
    elif early_send == 3:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        # Ischemic Stroke 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # TPA (5% gets TPA)
            if random() < TPA_decision_percentage:
                if random() <  TPA_consent_percentage:
                    # TPA long consent duration (90%, 10% don't take time)
                    if random() < TPA_long_consent_percentage:
                        print('%s TPA consent at %d' % (name, env.now))
                        yield env.timeout(TPA_consent_duration)
                        print('%s TPA consent ended at %d' % (name, env.now))

                    # TPA injection
                    with TPA_premixing.request() as req:
                        yield req
                        yield env.timeout(1)
                        print('%s TPA injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)

            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # IS DIDO times
            is_DIDO_times.append(door_out - door_in)

## EMS Patients Process

In [15]:
def EMS_patient(env, name, arrival_time, routine_care, TPA_premixing, ICU_duration, EMS_screen_percentage, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage):
    # Simulate patient arrivals to the PSC (door in)
    yield env.timeout(arrival_time)
    print('%s EMS arrival at %d' % (name, env.now))
    door_in = env.now
    
    # EMS screened as stroke? (83% yes, 17% no) if EMS didn't screen as stroke, then add triage, ed evaluation[, and maybe routine care]
    if random() > EMS_screen_percentage:
        # Triage
        print('%s Triage start at %d' % (name, env.now))
        yield env.timeout(triage_duration)
        print('%s Triage ended at %d' % (name, env.now))

        # ED Evaluation
        print('%s ED Evaluation start at %d' % (name, env.now))
        yield env.timeout(ED_eval_duration)
        print('%s ED Evaluation ended at %d' % (name, env.now))

        # Stroke identified? (90% yes, 10% no) if not, then add extra time in routine care
        if random() > stroke_identified_percentage:
            with routine_care.request() as req:
                yield req
                print('%s Routine Care at %d' % (name, env.now))
                yield env.timeout(routine_care_duration)
                print('%s Routine Care ended at %d' % (name, env.now))
        
    # Stroke Code Activation
    
    # Pre-mix TPA
    env.process(pharmacist(env, name, TPA_premixing, TPA_mixing_duration, pharmacist_arrival_duration))
    
    # early send at node 1 (after stroke code activation)
    if early_send == 1:
        # Ambulance Contact and Arrival
        print('%s Ambulance contact at %d' % (name, env.now))
        yield env.timeout(ambulance_arrival_duration)
        print('%s Ambulance arrived at %d' % (name, env.now))
        # CSC (door out)
        print('%s Patient send to CSC at %d' % (name, env.now))
        door_out = env.now
        print('%s DIDO:' % name, door_out - door_in)
        DIDO_times.append(door_out - door_in)
        
    # no early send
    elif early_send == 0:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # TPA (5% gets TPA)
            if random() < TPA_decision_percentage:
                if random() <  TPA_consent_percentage:
                    # TPA long consent duration (90%, 10% don't take time)
                    if random() < TPA_long_consent_percentage:
                        print('%s TPA consent at %d' % (name, env.now))
                        yield env.timeout(TPA_consent_duration)
                        print('%s TPA consent ended at %d' % (name, env.now))

                    # pharmacist in ED or not? yes if between 8am and 6pm
                    if ((env.now%1440)/60 > 8 and (env.now%1440)/60 < 18):
                        print('%s TPA mixing at %d' % (name, env.now))
                        yield env.timeout(TPA_mixing_duration)
                        print('%s TPA mixing ended and injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)
                    else:
                        print('%s TPA mixing at %d' % (name, env.now))
                        yield env.timeout(TPA_mixing_duration + pharmacist_arrival_duration)
                        print('%s TPA mixing ended and injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)
                    
                    # CTA decision (47% yes)
                    CTA = 0
                    if random() < CTA_decision_percentage:
                        CTA = 1
                        print('%s CTA Scan at %d' % (name, env.now))
                        yield env.timeout(CTA_duration)
                        print('%s CTA Scan ended at %d' % (name, env.now))

                    # transfer decision (15% yes)
                    if random() < transfer_decision_percentage:
                        if CTA:
                            # Ambulance Contact and Arrival
                            print('%s Ambulance contact at %d' % (name, env.now))
                            yield env.timeout(ambulance_arrival_duration)
                            print('%s Ambulance arrived at %d' % (name, env.now))

                        # CSC (door out)
                        print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in) 
                    else:
                        # ICU
                        door_out = env.now
                        print('%s ICU out time:' % name, door_out - door_in)
                        print('%s ICU at %d' % (name, env.now))
                        ICU_in = env.now
                        yield env.timeout(ICU_duration)
                        ICU_out = env.now
                        print('%s ICU ended at %d' % (name, env.now))  
                        ICU_times.append(ICU_out - ICU_in) 

                else: # (10% don't consent)
                    # Ambulance Contact and Arrival
                    print('%s Ambulance contact at %d' % (name, env.now))
                    yield env.timeout(ambulance_arrival_duration)
                    print('%s Ambulance arrived at %d' % (name, env.now))

                    # CSC (door out)
                    print('%s Patient send to CSC at %d' % (name, env.now))
                    door_out = env.now
                    print('%s DIDO:' % name, door_out - door_in)
                    DIDO_times.append(door_out - door_in)
    
    # early send at node 2 (after telestroke)
    elif early_send == 2:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # IS DIDO times
            is_DIDO_times.append(door_out - door_in)
            
    # early send at node 3 (after telestroke)
    elif early_send == 3:
        # CT Scan
        print('%s CT scan at %d' % (name, env.now))
        yield env.timeout(CT_duration)
        print('%s CT scan ended at %d' % (name, env.now))
        
        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            print('%s Telestroke ended at %d' % (name, env.now))

            # TPA (5% gets TPA)
            if random() < TPA_decision_percentage:
                if random() <  TPA_consent_percentage:
                    # TPA long consent duration (90%, 10% don't take time)
                    if random() < TPA_long_consent_percentage:
                        print('%s TPA consent at %d' % (name, env.now))
                        yield env.timeout(TPA_consent_duration)
                        print('%s TPA consent ended at %d' % (name, env.now))

                    # TPA injection
                    with TPA_premixing.request() as req:
                        yield req
                        yield env.timeout(1)
                        print('%s TPA injection at %d' % (name, env.now))
                        # door to needle time
                        door_to_needle.append(env.now - door_in)

            # Ambulance Contact and Arrival
            print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            print('%s Ambulance arrived at %d' % (name, env.now))

            # CSC (door out)
            print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # IS DIDO times
            is_DIDO_times.append(door_out - door_in)

# General ED Patients Process

In [5]:
def ED_patient(env, name, arrival_time, routine_care_duration):
    yield env.timeout(arrival_time)
    with routine_care.request() as req:
        yield req
        print('%s Routine Care at %d' % (name, env.now))
        yield env.timeout(routine_care_duration)
        print('%s Routine Care ended at %d' % (name, env.now))

## Data Input

In [16]:
path = './'
data = pd.read_csv(path + 'simulation_data.csv')
print('Previewing simulation data...')
data

Previewing simulation data...


,process,count,mean(mins),triangular,percentage,distribution/type,Notes
0,arrival_rate,NaN,2160.00,NaN,NaN,poisson,1.5 days on average per stroke patient
1,ICU_duration,NaN,60.00,NaN,NaN,exponential,NaN
2,EMS_arrivals_percentage,NaN,NaN,NaN,0.70,percentage,percentage of arrivals by EMS (not walk-in)
3,EMS_screen_percentage,NaN,NaN,NaN,0.83,percentage,NaN
4,triage_duration,NaN,6.00,NaN,NaN,exponential,NaN
5,ED_eval_duration,NaN,7.80,NaN,NaN,exponential,NaN
6,stroke_identified_percentage,NaN,NaN,NaN,0.90,percentage,NaN
7,routine_care_duration,NaN,90.00,NaN,NaN,exponential,NaN
8,CT_duration,NaN,28.33,NaN,NaN,exponential,NaN
9,HS_percentage,NaN,NaN,NaN,0.15,percentage,NaN


In [7]:
# ED arrival data for routine care
ED_arrival_data = pd.read_excel('./LFH ED arrival rate 2019.xlsx')
print('Previewing ED arrival data...')
print(ED_arrival_data.head())

# counting arrivals by date
ED_arrival_counts = {}
for index,row in ED_arrival_data.iterrows():
    month = str(row['Arrival Date/Time'].month)
    day = str(row['Arrival Date/Time'].day)
    hour = str(row['Arrival Date/Time'].hour)
    date_key = month + '-' + day + '-' + hour
    if date_key not in ED_arrival_counts.keys():
        ED_arrival_counts[date_key] = 1
    else:
        ED_arrival_counts[date_key] += 1

# organizing counts by hour
import re
pattern = '.+-.+-(.+)'
ED_hour_counts = {}
for key, value in ED_arrival_counts.items():
    hour = re.findall(pattern,key)[0]
    if hour not in ED_hour_counts.keys():
        ED_hour_counts[hour] = [value]
    else:
        ED_hour_counts[hour].append(value)
        
# average arrival rate per hour for general ED patient arrivals to routine care
avg_arrivals_per_hour = [0]*24
for key, value in ED_hour_counts.items():
    idx = int(key)
    avg_arrivals_per_hour[idx] = np.mean(value)
# arrival rates in minutes for each hour
time_varying = [60/arrival for arrival in avg_arrivals_per_hour]

Previewing ED arrival data...
    Arrival Date/Time  Month  Year Site
0 2019-01-01 01:56:00      1  2019  LFH
1 2019-01-01 05:21:00      1  2019  LFH
2 2019-01-01 07:35:00      1  2019  LFH
3 2019-01-01 10:36:00      1  2019  LFH
4 2019-01-01 11:56:00      1  2019  LFH


## Simulation

In [18]:
early_send = int(input('Send early to CSC? (Please enter node number to send early at (1,2,3))\nOr enter 0 for no early sending '))
seed(333555)
DIDO_times = []
hs_DIDO_times = []
is_DIDO_times = []
ICU_times = []
door_to_needle = []

TPA_mixing_duration_triangular = [float(x) for x in data[data.process =='TPA_mixing_duration']['triangular'].item().split(',')]
pharmacist_arrival_duration_triangular = [float(x) for x in data[data.process =='pharmacist_arrival_duration']['triangular'].item().split(',')]

# define environment
env = simpy.Environment()
# define ICU capacity
routine_care = simpy.Resource(env, capacity=int(data[data.process =='routine_care_capacity']['count']))
# define total number of stroke patients
num_patients = data[data.process =='num_stroke_patients']['count']
# pharmacist premixing TPA
TPA_premixing = []
for i in range(int(num_patients)): 
    TPA_premixing.append(simpy.Resource(env, capacity=1))

# general ED arrivals
ED_arrival_time = 0
for i in range(100):
    arrival_rate = np.random.poisson(time_varying[math.floor(ED_arrival_time%1440/60)])
    ED_arrival_time += arrival_rate
    routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
    env.process(ED_patient(env,'ED_Patient %d' % i, ED_arrival_time, routine_care_duration))

# EMS arrivals
EMS_arrival_time = 0
for i in range(int(float(data[data.process =='EMS_arrivals_percentage']['percentage'])*int(num_patients))):
    ICU_duration = expovariate(1/float(data[data.process =='ICU_duration']['mean(mins)']))
    EMS_screen_percentage = data[data.process =='EMS_screen_percentage']['percentage'].item()
    triage_duration = expovariate(1/float(data[data.process =='triage_duration']['mean(mins)']))
    ED_eval_duration = expovariate(1/float(data[data.process =='ED_eval_duration']['mean(mins)']))
    stroke_identified_percentage = data[data.process =='stroke_identified_percentage']['percentage'].item()
    routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
    CT_duration = expovariate(1/float(data[data.process =='CT_duration']['mean(mins)']))
    HS_percentage = data[data.process =='HS_percentage']['percentage'].item()
    ambulance_arrival_duration = expovariate(1/float(data[data.process =='ambulance_arrival_duration']['mean(mins)']))
    telestroke_duration = expovariate(1/float(data[data.process =='telestroke_duration']['mean(mins)']))
    TPA_decision_percentage = data[data.process =='TPA_decision_percentage']['percentage'].item()
    TPA_consent_percentage = data[data.process =='TPA_consent_percentage']['percentage'].item()
    TPA_long_consent_percentage = data[data.process =='TPA_long_consent_percentage']['percentage'].item()
    TPA_consent_duration = expovariate(1/float(data[data.process =='TPA_consent_duration']['mean(mins)']))
    TPA_mixing_duration = triangular(TPA_mixing_duration_triangular[0],TPA_mixing_duration_triangular[1],TPA_mixing_duration_triangular[2])
    pharmacist_arrival_duration = triangular(pharmacist_arrival_duration_triangular[0],pharmacist_arrival_duration_triangular[1],pharmacist_arrival_duration_triangular[2])
    CTA_decision_percentage = data[data.process =='CTA_decision_percentage']['percentage'].item()
    CTA_duration = expovariate(1/float(data[data.process =='CTA_duration']['mean(mins)']))
    transfer_decision_percentage = data[data.process =='transfer_decision_percentage']['percentage'].item()
    
    # arrival rate
    EMS_arrival_rate = np.random.poisson(float(data[data.process =='arrival_rate']['mean(mins)']))
    EMS_arrival_time += EMS_arrival_rate
    env.process(EMS_patient(env, 'EMS_Patient %d' % i, EMS_arrival_time, routine_care, TPA_premixing[i], ICU_duration, EMS_screen_percentage, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage))

# Walkin arrivals
walkin_arrival_time = 0
for i in range(int(float(data[data.process =='EMS_arrivals_percentage']['percentage'])*int(num_patients)),int(num_patients)):
    ICU_duration = expovariate(1/float(data[data.process =='ICU_duration']['mean(mins)']))
    EMS_screen_percentage = data[data.process =='EMS_screen_percentage']['percentage'].item()
    triage_duration = expovariate(1/float(data[data.process =='triage_duration']['mean(mins)']))
    ED_eval_duration = expovariate(1/float(data[data.process =='ED_eval_duration']['mean(mins)']))
    stroke_identified_percentage = data[data.process =='stroke_identified_percentage']['percentage'].item()
    routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
    CT_duration = expovariate(1/float(data[data.process =='CT_duration']['mean(mins)']))
    HS_percentage = data[data.process =='HS_percentage']['percentage'].item()
    ambulance_arrival_duration = expovariate(1/float(data[data.process =='ambulance_arrival_duration']['mean(mins)']))
    telestroke_duration = expovariate(1/float(data[data.process =='telestroke_duration']['mean(mins)']))
    TPA_decision_percentage = data[data.process =='TPA_decision_percentage']['percentage'].item()
    TPA_consent_percentage = data[data.process =='TPA_consent_percentage']['percentage'].item()
    TPA_long_consent_percentage = data[data.process =='TPA_long_consent_percentage']['percentage'].item()
    TPA_consent_duration = expovariate(1/float(data[data.process =='TPA_consent_duration']['mean(mins)']))
    TPA_mixing_duration = triangular(TPA_mixing_duration_triangular[0],TPA_mixing_duration_triangular[1],TPA_mixing_duration_triangular[2])
    pharmacist_arrival_duration = triangular(pharmacist_arrival_duration_triangular[0],pharmacist_arrival_duration_triangular[1],pharmacist_arrival_duration_triangular[2])
    CTA_decision_percentage = data[data.process =='CTA_decision_percentage']['percentage'].item()
    CTA_duration = expovariate(1/float(data[data.process =='CTA_duration']['mean(mins)']))
    transfer_decision_percentage = data[data.process =='transfer_decision_percentage']['percentage'].item()
    
    # arrival rate
    walkin_arrival_rate = np.random.poisson(float(data[data.process =='arrival_rate']['mean(mins)']))
    walkin_arrival_time += walkin_arrival_rate
    env.process(walkin_patient(env, 'walkin_Patient %d' % i, walkin_arrival_time, routine_care, TPA_premixing[i], ICU_duration, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage))
        
env.run(until=3000*num_patients)
print("\nCSC number of patients sent:",len(DIDO_times),"(%d percent)" % (len(DIDO_times)/int(num_patients)*100))
print("Mean DIDO time:", np.mean(DIDO_times), "minutes")
if early_send == 0 or 2:
    if early_send != 2 and early_send != 3:
        print("ICU number of patients sent:",len(ICU_times),"(%d percent)" % (len(ICU_times)/int(num_patients)*100))
        print("Mean ICU out time", np.mean(ICU_times), "minutes")
    print("\nMean HS DIDO time:", np.mean(hs_DIDO_times), "minutes (%d patients)" % len(hs_DIDO_times))
    print("Mean IS DIDO time:", np.mean(is_DIDO_times), "minutes (%d patients)"% len(is_DIDO_times))
    if early_send != 2:
        print("Mean door-to-needle time:", np.mean(door_to_needle), "minutes (%d patients)"% len(door_to_needle))

Send early to CSC? (Please enter node number to send early at (1,2,3))
Or enter 0 for no early sending 0
ED_Patient 0 Routine Care at 49
ED_Patient 0 Routine Care ended at 72
ED_Patient 1 Routine Care at 99
ED_Patient 2 Routine Care at 149
ED_Patient 2 Routine Care ended at 149
ED_Patient 3 Routine Care at 195
ED_Patient 1 Routine Care ended at 243
ED_Patient 4 Routine Care at 251
ED_Patient 5 Routine Care at 295
ED_Patient 6 Routine Care at 350
ED_Patient 3 Routine Care ended at 355
ED_Patient 7 Routine Care at 387
ED_Patient 6 Routine Care ended at 396
ED_Patient 4 Routine Care ended at 435
ED_Patient 8 Routine Care at 451
ED_Patient 8 Routine Care ended at 461
ED_Patient 5 Routine Care ended at 469
ED_Patient 9 Routine Care at 498
ED_Patient 9 Routine Care ended at 538
ED_Patient 10 Routine Care at 544
ED_Patient 7 Routine Care ended at 595
ED_Patient 11 Routine Care at 607
ED_Patient 10 Routine Care ended at 627
ED_Patient 12 Routine Care at 652
ED_Patient 12 Routine Care ended at 